In [8]:
OFFLINE = True

In [ ]:
import os, json, time
import gc
from IPython.display import display, Markdown

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import transformers
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM, TrainingArguments
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores import FAISS, Chroma
# Text embedding / Texxt Splitter for RAG 
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from datasets import Dataset, DatasetDict, load_dataset

# Adv RAG library 

# for evaluate  LLM 
import evaluate # require online 
# from deepeval.metrics import GEval
# from deepeval.test_case import LLMTestCase
# from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric , ContextualRelevancyMetric, ContextualRecallMetric
import pytest
# import trulens

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [7]:
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        gc.collect()
        time.sleep(0.3)

In [ ]:
# Enable/Disable Function
FEW_SHOT_TEST= False#True
USE_RAG = True#False#False #True#True
USE_WANDB = False#True # for  LLM evalution and debug , track fine tuning performance
USE_TRULENS = False # for LLM evalution
USE_DEEPEVAL = False # for LLM evalution   (require openAI API key)
USE_TRAIN =  True #True #False#True 
USE_INFER =  False # for submision prediction only , no test model 
if OFFLINE :
    USE_WANDB = False # Wandb only support online  
if device.type == "cpu": #requred GPU support for fine turning 
    USE_TRAIN= False

if USE_WANDB:
    # train report to  W&B tool
    import wandb
    from kaggle_secrets import UserSecretsClient
    reportTo= "wandb"
    user_secrets = UserSecretsClient()
    my_secret = user_secrets.get_secret("wandb_api_key") 
    wandb.login(key=my_secret) # login 
else: 
    reportTo = "none"# None
#     os.environ["WANDB_DISABLED"] = True#“true”
    

In [ ]:
if USE_TRULENS:
    from trulens_eval import Tru
    tru = Tru()
    tru.reset_database()

In [ ]:
reportTo


In [ ]:
device.type


In [ ]:
if device.type == "cuda" and USE_TRAIN == True: #requred GPU support
    # for LoRA fine tuning
    from trl import SFTTrainer
    from peft import LoraConfig, PeftModel

In [ ]:
sampleSubmitFile = "/kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv"
trainFile = "/kaggle/input/ai-mathematical-olympiad-prize/train.csv"
testFile = "/kaggle/input/ai-mathematical-olympiad-prize/test.csv"
mathQSATrainFile = "/kaggle/input/math-qsa-dataset/train.csv"
mathQSATestFile = "/kaggle/input/math-qsa-dataset/test.csv"
gsm8kTrainFile = "/kaggle/input/gsm8k-grade-school-math-8k-dataset-for-llm/gsm8k/main/train-00000-of-00001.parquet"
gsm8kTestFile = "/kaggle/input/gsm8k-grade-school-math-8k-dataset-for-llm/gsm8k/main/test-00000-of-00001.parquet"
mathQATrainFile = "/kaggle/input/math-qa-for-aqua-rat-dataset/MathQA/train.json"
mathQATestFile = "/kaggle/input/math-qa-for-aqua-rat-dataset/MathQA/test.json"
orcaMath200kFile = "/kaggle/input/microsoftorca-math-word-problems-200k/orca-math-word-problems-200k/data/train-00000-of-00001.parquet"

In [ ]:
clearMemory()


In [ ]:
trainDF = pd.read_csv(trainFile)
trainDF

In [ ]:
trainDF.describe()


In [ ]:
trainDF.iloc[7]["answer"]


In [ ]:
testDF = pd.read_csv(testFile)
testDF.info()

In [ ]:
trainQSADF = pd.read_csv(mathQSATrainFile)
trainQSADF.head()

In [ ]:
trainQSADF["problem"][0]


In [ ]:
trainQSADF.describe()


In [ ]:
trainQADF= pd.read_json(mathQATrainFile)
trainQADF.head()

In [ ]:
trainQADF["options"][5]


In [ ]:
trainGSM8KDF =pd.read_parquet(gsm8kTrainFile)
trainGSM8KDF.head()


In [ ]:
trainGSM8KDF.info()


In [ ]:
trainGSM8KDF.iloc[0]["answer"]


In [ ]:
testGSM8KDF = pd.read_parquet(gsm8kTestFile)
testGSM8KDF.head()

In [ ]:
testGSM8KDF["answer"][12]


In [ ]:
trainOracMath200kDF = pd.read_parquet(orcaMath200kFile)
trainOracMath200kDF.head()

In [ ]:
trainOracMath200kDF["answer"].iloc[1]


In [ ]:
trainDF["problem"][9]


In [ ]:
## cleaning data set
# trainDF["problem"] = trainDF["problem"].str.replace("$", '')
# trainDF["problem"] = trainDF["problem"].str.replace("\\vert", '|')
# trainDF["problem"] = trainDF["problem"].str.replace("\\left", '')
# trainDF["problem"] = trainDF["problem"].str.replace("\\right", '')
# trainDF["problem"] = trainDF["problem"].str.replace("\\mathbb", '')

In [ ]:
print(trainDF["problem"][1])


In [ ]:
print(trainDF["answer"][1])
